In [104]:
import bw2data as bd

bd.projects.current

'nonlinear-method-test'

In [105]:
try:
    bd.projects.delete_project("nonlinear-method-test", True)
except ValueError:
    print("project does not exist")

In [106]:
bd.projects.set_current("nonlinear-method-test")

In [107]:
db = bd.Database("db")
# Let the metadata system know this database exists. Not necessary if using a `bw2io` importer.
db.register()

![](/home/ra/projects/enbios/enbios/experiment/non_linear_poc.png)

In [108]:
final_product = db.new_node(
    code='product',
    name='random production',
    location='ES',
    unit='unit'
)
final_product.save()

cf = db.new_node(
    code='cf',
    name='carbon fibre production',
    location='NO',
    unit='kg'
)
cf.save()

ng = db.new_node(
    code='ng',
    name='natural gas production',
    location='NO',
    unit='MJ'
)
ng.save()

steel = db.new_node(
    code='steel',
    name='steel production',
    location='DE',
    unit='kg'
)
steel.save()

In [109]:
co2 = db.new_node(
    code='co2',
    name="Carbon Dioxide",
    categories=('air',),
    type='emission',
    unit='kg'
)

co2.save()

bio_chem = db.new_node(
    code='bio_chem',
    name="Biochemicals",
    categories=('air',),
    type='emission',
    unit='kg'
)

bio_chem.save()

In [110]:
def create_edge(act, input, amount, type):
    return act.new_edge(
        amount=amount,
        type=type,
        input=input
    ).save()


TECHNOSPHERE = 'technosphere'
BIOSPHERE = 'biosphere'

create_edge(final_product, cf, 1, TECHNOSPHERE)
create_edge(final_product, steel, 2, TECHNOSPHERE)

create_edge(cf, co2, 25, BIOSPHERE)
create_edge(cf, ng, 100, TECHNOSPHERE)
create_edge(cf, bio_chem, 10, BIOSPHERE)

create_edge(steel, bio_chem, 5, BIOSPHERE)
create_edge(ng, bio_chem, 1, BIOSPHERE)

In [111]:
ipcc = bd.Method(('IPCC',))
ipcc.write([
    (co2.key, {'amount': 1}),
    (bio_chem.key, {'amount': 2})
])

In [112]:
import bw2calc
lca = bw2calc.LCA(demand={final_product: 1}, method=('IPCC',))
lca.lci()
lca.lcia()
lca.score


265.0

note that we actually dont need to pass a method, in order to do the lci, because we are not doing any lcia, when doing non-linear calculations

In [113]:
lca.supply_array

array([  1.,   1., 100.,   2.])

In [114]:
# this is the most important matrix
lca.inventory.toarray()

array([[  0.,  25.,   0.,   0.],
       [  0.,  10., 100.,  10.]])

In [115]:
# lca.biosphere_matrix.toarray()

In [116]:
# not sure exactly why it has this shape
lca.characterization_matrix.toarray()

array([[1., 0.],
       [0., 2.]])

In [117]:
# this is how lcia is done normally
char_inv = lca.characterization_matrix * lca.inventory.toarray()
char_inv, f"SCORE: {char_inv.sum()}"

(array([[  0.,  25.,   0.,   0.],
        [  0.,  20., 200.,  20.]]),
 'SCORE: 265.0')

In [118]:
# we can also sum up only one axis (which is, summing each individual bioflow)
# this is the starting point for the nonlinear calculations
# Term candidates: Total Biosphere Demand Vector, Aggregated Environmental Impact Vector, Environmental Load Vector
summed_inventory = lca.inventory.sum(1)
summed_inventory

matrix([[ 25.],
        [120.]])

In [119]:
# this allows still the same matrix multiplication, because the number of rows is the same
(lca.characterization_matrix * summed_inventory).sum()

265.0

In [120]:
# lets make a test of arbitrary functions, but use the linear values initially
import numpy as np
cm = lca.characterization_matrix.sum(0)
lin_values = np.squeeze(np.asarray(cm))
# the characterisation array
lin_values 

array([1., 2.])

In [123]:
# this looks strange, but we need v=value, because of python closures 
funcs = []
for value in lin_values:
  f = lambda x, v=value: x * v
  funcs.append(f)
# what we get now is a list of functions, each of them is linear with the same coefficient as in the characterisation matrix
print(funcs)
print(funcs[0](1))
print(funcs[1](1))

[<function <lambda> at 0x7f825d34be20>, <function <lambda> at 0x7f825d349990>]
1.0
2.0


In [124]:
# lets test it...
result = 0.0
for summed_row, function in zip(summed_inventory, funcs):
    print(summed_row, function(summed_row))
    result += function(summed_row)
print(result.sum()) # voilà

[[25.]] [[25.]]
[[120.]] [[240.]]
265.0


In [61]:
import numpy as np
import bw2calc

# create a LCA object
lca = bw2calc.LCA(demand={final_product: 1})

# calculate the lci
lca.lci()

# get the summed inventory, which are basically all biosphere activities outputs
summed_inventory = lca.inventory.sum(1)

# example: 2 arbitrary, independent functions:
funcs = [
    lambda x: x*x+2,
    lambda x: (x/3)-3
]

# result value for each biosphere activity
result = np.array([])
for summed_row, function in zip(summed_inventory, funcs):
    result = np.append(result, function(summed_row))
    
# non-linear impact of each biosphere activity
print(result)
# result
print(result.sum())

# same, but directly to a single value
result = 0.0
for summed_row, function in zip(summed_inventory, funcs):
    result += function(summed_row)
print(result.sum())

[627.  37.]
664.0
664.0


In [62]:
# since the example is very simple, we can also do it like this, to show the correctness
(funcs[0](summed_inventory[0]) + funcs[1](summed_inventory[1])).sum()

664.0

In [102]:
# appendix: how to get the biosphere activities in order in the matrix, so we can to specify the functions in the same order
from bw2data.backends import Activity, ActivityDataset

ids = []
for index, activity_id in sorted(lca.dicts.biosphere.reversed.items()):
    # print(index, activity_id)
    # act = ActivityDataset.get_or_none(ActivityDataset.id == activity_id)
    # print(act.code, act.name)
    ids.append(activity_id)
    
activities = ActivityDataset.select().where(ActivityDataset.id.in_(ids))
sorted_activities = sorted(activities, key=lambda activity: ids.index(activity.id))
# sorted_activities
for activity in sorted_activities:
    print(activity.id, activity.code, activity.name)

5 co2 Carbon Dioxide
6 bio_chem Biochemicals
